# 01 — Full Pipeline Validation: PyTorch → Quantize → NANO-RUST-AI

`Conv2D(1→4, 3×3) → ReLU → Flatten → Dense(144→10)` with scale-aware requantization

In [1]:
from _setup import setup_all, PROJECT_ROOT
setup_all()

   NANO-RUST Setup
✅ Virtualenv: C:\Users\suoya\OneDrive\Documents\nanorust\.venv
✅ numpy
✅ torch
✅ maturin

✅ rustc 1.93.1 (01f6ddf75 2026-02-11)

🔧 CARGO_TARGET_DIR = C:\Users\suoya\.nanorust_target
🔨 Building nano-rust-py (maturin develop --release)...
✅ Build OK (5.0s)
       Finished `release` profile [optimized] target(s) in 0.03s
   📦 Built wheel for CPython 3.13 to C:\Users\suoya\AppData\Local\Temp\.tmpT6b4lD\nano_rust_py-0.1.0-cp313-cp313-win_amd64.whl

🎉 Setup complete! Ready to use.


In [2]:
import numpy as np
import torch
import torch.nn as nn
from nano_rust_utils import quantize_to_i8, quantize_weights, calibrate_model
import nano_rust_py
print('✅ All modules loaded')

✅ All modules loaded


In [3]:
torch.manual_seed(42)
model = nn.Sequential(
    nn.Conv2d(1, 4, kernel_size=3, stride=1, padding=0, bias=True),
    nn.ReLU(),
    nn.Flatten(),
    nn.Linear(4 * 6 * 6, 10, bias=True),
)
model.eval()
print(model)
print(f'Params: {sum(p.numel() for p in model.parameters()):,}')

Sequential(
  (0): Conv2d(1, 4, kernel_size=(3, 3), stride=(1, 1))
  (1): ReLU()
  (2): Flatten(start_dim=1, end_dim=-1)
  (3): Linear(in_features=144, out_features=10, bias=True)
)
Params: 1,490


In [4]:
# Quantize weights and calibrate
q_weights = quantize_weights(model)

torch.manual_seed(42)
input_tensor = torch.randn(1, 1, 8, 8)
q_input, input_scale = quantize_to_i8(input_tensor.numpy().flatten())

requant = calibrate_model(model, input_tensor, q_weights, input_scale)
print('=== Calibrated Requant Params ===')
for name, (m, s, bc) in requant.items():
    print(f'  Layer {name}: M={m}, shift={s}, bias_corrected={bc}')

=== Calibrated Requant Params ===
  Layer 0: M=6190084, shift=31, bias_corrected=[3, 18, 2, -7]
  Layer 3: M=5161138, shift=31, bias_corrected=[-2, 11, -14, -13, -5, -7, 8, 14, -2, 19]


In [5]:
# PyTorch reference
with torch.no_grad():
    pytorch_output = model(input_tensor).numpy().flatten()

# NANO-RUST with calibrated requant + corrected bias
nano = nano_rust_py.PySequentialModel(input_shape=[1, 8, 8], arena_size=32768)

# Conv2D: use corrected bias from calibration
conv_m, conv_s, conv_bias = requant['0']
nano.add_conv2d_with_requant(
    q_weights['0']['weights'].flatten().tolist(),
    conv_bias,  # bias in output scale S_y, not independent S_b
    1, 4, 3, 3, 1, 0, conv_m, conv_s)
nano.add_relu()
nano.add_flatten()

# Dense: use corrected bias from calibration
dense_m, dense_s, dense_bias = requant['3']
nano.add_dense_with_requant(
    q_weights['3']['weights'].flatten().tolist(),
    dense_bias,  # bias in output scale
    dense_m, dense_s)

nano_output = nano.forward(q_input.tolist())

# Compare
q_pytorch, _ = quantize_to_i8(pytorch_output)
nano_arr = np.array(nano_output, dtype=np.int8)
diff = np.abs(q_pytorch.astype(np.int32) - nano_arr.astype(np.int32))

TOLERANCE = 20
print('=' * 50)
print('       PIPELINE VALIDATION RESULTS')
print('=' * 50)
print(f'PyTorch (i8): {q_pytorch.tolist()}')
print(f'NANO-RUST:    {nano_arr.tolist()}')
print(f'Max diff:     {int(np.max(diff))}')
print(f'Mean diff:    {float(np.mean(diff)):.2f}')
print(f'PyTorch class:  {int(np.argmax(q_pytorch))}')
print(f'NANO-RUST class:{int(np.argmax(nano_arr))}')
print('-' * 50)
ok = int(np.max(diff)) <= TOLERANCE
print(f'{"✅ Numerical: PASS" if ok else "❌ Numerical: FAIL"} (tolerance={TOLERANCE})')
match = int(np.argmax(q_pytorch)) == int(np.argmax(nano_arr))
print(f'{"✅ Classification: AGREE" if match else "⚠️ Classification: DISAGREE"}')
print('=' * 50)
print('🎉 PASSED!' if ok else '⚠️ FAILED')

       PIPELINE VALIDATION RESULTS
PyTorch (i8): [-38, -33, 5, -11, -101, -31, 12, 127, 122, -2]
NANO-RUST:    [-35, -32, 7, -11, -102, -29, 13, 126, 119, -3]
Max diff:     3
Mean diff:    1.50
PyTorch class:  7
NANO-RUST class:7
--------------------------------------------------
✅ Numerical: PASS (tolerance=20)
✅ Classification: AGREE
🎉 PASSED!
